In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as c
from astropy.wcs import WCS
import astropy.coordinates as coord
import astropy.units as u
from pyutils import *
import astropy.io.fits as fits
import healpy as hp
from astropy.table import Table,join

%load_ext autoreload
%autoreload 2

In [ ]:
ROOT_FOLDER = "../bin/"
BGS_PREVIOUS_FOLDER = "../bin/BGS v1.0/"
OLD_FILE = ROOT_FOLDER + "BGS_BRIGHT_full.dat.fits"
FILE = ROOT_FOLDER + "BGS_ANY_full.dat.fits"
RAND_FILE = ROOT_FOLDER + "BGS_BRIGHT_0_full.ran.fits"

PROB_OBS_FILE = ROOT_FOLDER + "mainbw-bright-allTiles_v1.fits"

# TODO ensure this is right
def get_app_mag(flux):
    """This converts nanomaggies into Pogson magnitudes"""
    return 22.5 - 2.5*np.log10(flux)

## Healpix

In [ ]:
BGS_N = hp.read_map(ROOT_FOLDER + 'BGS_BRIGHT_mapprops_healpix_nested_nside256_N.fits')
BGS_S = hp.read_map(ROOT_FOLDER + 'BGS_BRIGHT_mapprops_healpix_nested_nside256_S.fits')

In [ ]:
hp.mollview(
    BGS_N,
    coord=["G", "E"],
    title="Histogram equalized Ecliptic",
    unit="Galaxies",
    norm="hist",
    min=0,
    max=1,
)
hp.graticule()

In [ ]:
hp.mollview(
    BGS_S,
    coord=["G", "E"],
    title="Histogram equalized Ecliptic",
    unit="Galaxies",
    norm="hist",
    min=0,
    max=1,
)
hp.graticule()

## Main File

   We need:

   'app_mag' CALCULATE IT from FLUX_R

   'dec' GOT IT

   'g_r' GOT IT (but k-corr?)

   'ra' GOT IT

   'z_obs' GOT IT

   If ZWARN say multi fits (4), consider it LOST. Use DELTACHI2 < 40 instead of ZWARN (higher bar for good fit) We can actually use that info later if available.

3 pass info is in the NTILE column.

In [ ]:
u_table = Table.read(FILE, format='fits')


In [ ]:
p_table = Table.read(PROB_OBS_FILE, format='fits')
j_table = join(u_table, p_table, keys="TARGETID")

In [ ]:
old_table = Table.read(OLD_FILE, format='fits')


In [ ]:
np.setxor1d(u_table.colnames, old_table.colnames)

In [ ]:
u_table.colnames

In [ ]:
print(len(u_table['TARGETID']))
print(len(j_table['TARGETID']))
print(len(p_table['TARGETID']))

In [ ]:
plt.hist(u_table['Z_not4clus'], bins=50)
plt.title("Z_not4clus")
plt.yscale('log')
print(np.min(u_table['Z_not4clus']), np.max(u_table['Z_not4clus']))
print(u_table['Z_not4clus'].mask)

In [ ]:
print(np.unique(u_table['ZWARN']))
print(np.unique(u_table['ZWARN_MTL']))
print(np.unique(u_table['SPECTYPE']))
print(np.unique(u_table['NTILE']))
print(np.unique(u_table['TARGET_STATE']))

In [ ]:
u_table.add_index(['TARGETID'])

### Get the galaxy data we actually need

In [ ]:
table = u_table

APP_MAG_CUT = 22.0
Z_MIN = 0.01
Z_MAX = 0.8
obj_type = table['SPECTYPE'].data.data
dec = table['DEC']
ra = table['RA']
z_obs = table['Z_not4clus'].data.data
target_id = table['TARGETID']
flux_r = table['FLUX_R']
flux_g = table['FLUX_G']
app_mag = get_app_mag(table['FLUX_R'])
app_mag_g = get_app_mag(table['FLUX_G'])
#g_r = app_mag - app_mag_g
sdss_g_r = table['ABSMAG_SDSS_G'] - table['ABSMAG_SDSS_R'] 
#p_obs = table['PROB_OBS'] 
unobserved = table['ZWARN'] == 999999
deltachi2 = table['DELTACHI2'].data.data
ntiles = table['NTILE']
abs_mag_sdss = table['ABSMAG_SDSS_R']

before_count = len(dec)
print(before_count, "objects in FITS file")

# Make filter array (True/False values)
galaxy_filter = np.logical_or(obj_type == b'GALAXY', obj_type == b'')
galaxy_observed_filter = obj_type == b'GALAXY'
app_mag_filter = app_mag < APP_MAG_CUT
redshift_filter = z_obs > Z_MIN
redshift_hi_filter = z_obs < Z_MAX
deltachi2_filter = deltachi2 > 40
abs_mag_sdss_filter = abs_mag_sdss < 100

keep = np.all([galaxy_observed_filter, app_mag_filter, redshift_filter, redshift_hi_filter, deltachi2_filter, abs_mag_sdss_filter], axis=0)
keep_3 = np.all([keep, ntiles >= 3], axis=0)
unobserved_3 = np.all([ntiles >= 3, unobserved], axis=0)
treat_as_unobserved_3 = np.all([galaxy_observed_filter, app_mag_filter, ntiles >= 3, np.invert(deltachi2_filter)], axis=0)
eff_unobserved_3 = np.logical_or(unobserved_3, treat_as_unobserved_3)

print(f"There are {len(table['DEC'][galaxy_filter])} galaxy targets in the sample,of which {len(table['DEC'][galaxy_observed_filter])} are observed. Of these, {len(table['DEC'][keep])} are in the bright (<{APP_MAG_CUT} mag) sample and pass our quality of z-fit checks.")
print(f"Of those, only {len(table['DEC'][keep_3])} are in the 3-pass coverage area.")
print(f"There are {len(table['DEC'][unobserved_3])} unobserved galaxies in the 3-pass coverage area.")
print(f"There are also {len(table['DEC'][treat_as_unobserved_3])} bad observed galaxies in the 3pass regionthat we will treat as unobserved, making a combined effective total of {len(table['DEC'][eff_unobserved_3])} 3pass unobserved galaxies.")

obj_type = obj_type[keep_3]
dec = dec[keep_3]
ra = ra[keep_3]
z_obs = z_obs[keep_3]
target_id = target_id[keep_3]
flux_r = flux_r[keep_3]
app_mag = app_mag[keep_3]
app_mag_g = app_mag_g[keep_3]
#p_obs = p_obs[keep_3]
unobserved = unobserved[keep_3]
deltachi2 = deltachi2[keep_3]
ntiles = ntiles[keep_3]
abs_mag_sdss = abs_mag_sdss[keep_3]
sdss_g_r = sdss_g_r[keep_3]

after_count = len(dec)
g_r = app_mag_to_abs_mag(app_mag_g, z_obs) - app_mag_to_abs_mag(app_mag, z_obs) # TODO is this right

print(f"\nAfter all filters we have {after_count} of the original {before_count} rows.")

#print("Frac area after all filters", estimate_frac_area(ra, dec))
#fig=make_map(ra, dec)


In [ ]:
# TODO why is this not bimodal? 
# Maybe I don't go to app mags right...
junk=plt.hist(g_r, bins=300, alpha=0.5)
junk=plt.hist(sdss_g_r, bins=300, alpha=0.5)

In [ ]:
app_mag_new = app_mag

In [ ]:
max(app_mag_new)

In [ ]:
plt.hist(app_mag_new, label="my abs_mag", bins=100, alpha=0.5)
plt.hist(app_mag_old, label="my abs_mag", bins=100, alpha=0.5)
plt.xlabel("Apparent Mag")
plt.ylabel("Count")
#plt.yscale('log')
plt.legend()


In [ ]:
mag1 = abs_mag_sdss
mag2 = app_mag_to_abs_mag(get_app_mag(app_mag), z_obs)

In [ ]:
print(len(mag1), len(mag2))

In [ ]:
# Compare Absolute Magnitudes
bins = np.linspace(-25, -10, 100)
my_counts, my_bins, my_p = plt.hist(mag2, label="my abs_mag", bins=bins, alpha=0.5)
alex_counts, alex_bins, alex_p = plt.hist(mag1, label="ABSMAG_SDSS_R", bins=bins, alpha=0.5)
plt.xlabel("Absolute Mag")
plt.ylabel("Count")
plt.title("Compare Absolute Mags")
#plt.yscale('log')
plt.legend()

print(f"The peak is shifted from ABSMAG_SDSS_R {alex_bins[np.argmax(alex_counts)]:.1f} to my {my_bins[np.argmax(my_counts)]:.1f}")


In [ ]:
fig=make_map(ra, dec)

In [ ]:
fig=make_map(ra[5], dec[5], alpha=1)

In [ ]:
plt.hist(obj_type)

In [ ]:
plt.figure(figsize=(32,8))
plt.hist(u_table['TARGET_STATE'])
plt.title('Target State')
plt.yscale('log')

In [ ]:
print(np.min(z_obs))
print(np.max(z_obs))

In [ ]:
u_table['Z_not4clus'].mask[0:10]

In [ ]:
u_table['ZWARN'][0:10]

In [ ]:
plt.hist(u_table['ZWARN'])
plt.title('ZWARN')
plt.yscale('log')

In [ ]:
plt.hist(u_table['ZWARN_MTL'])
plt.title('ZWARN_MTL')
plt.yscale('log')

In [ ]:
plt.hist(z_obs, bins=50)
plt.title('Redshift')
plt.yscale('log')

In [ ]:
plt.hist(p_obs, bins=50)
plt.title('Probabilty to Observe')

In [ ]:
plt.hist(app_mag, bins=50)
plt.yscale('log')

In [ ]:
plt.hist(get_app_mag(u_table['FLUX_G']) - get_app_mag(u_table['FLUX_R']) , bins=20)
plt.title('g-r')
plt.yscale('log')

### Randoms Analysis for Footprint

In [ ]:
rtable = Table.read(RAND_FILE, format='fits')


In [ ]:
for c in rtable.colnames:
    if u_table.colnames.__contains__(c):
        print(f"Column {c} is in both files")

In [ ]:
r_dec = rtable['DEC']
r_ra = rtable['RA']
r_ntiles = rtable['NTILE']

RANDOMS_DENSITY = 2500 # per square degree

onepass_footprint = len(r_dec) / RANDOMS_DENSITY # in degrees squared
onepass_frac_area = onepass_footprint / DEGREES_ON_SPHERE

three_pass_filter = r_ntiles >= 3 # 3pass coverage
r_dec3 = r_dec[three_pass_filter]
r_ra3 = r_ra[three_pass_filter]

threepass_footprint = len(r_dec3) / RANDOMS_DENSITY # in degrees squared
threepass_frac_area = threepass_footprint / DEGREES_ON_SPHERE

In [ ]:
print(f"BGS Y1 1pass Footprint calculated from randoms is {onepass_footprint} square degrees or frac_area={onepass_frac_area}")
print(f"BGS Y1 3pass Footprint calculated from randoms is {threepass_footprint} square degrees or frac_area={threepass_frac_area}")